In [19]:
# reset specific variables (replace regular_expression by the variables of interest)
#%reset_selective <regular_expression>

# reset all variables
%reset -f

In [1]:
# Importing libraries

from dask import dataframe as dd
from datetime import datetime, date, timedelta
from pathlib import Path
import matplotlib.pyplot as plt
from pylab import savefig
import seaborn as sns
import pandas as pd
import csv
%matplotlib inline

In [2]:
# Loading datasaet - AlibabaOverTime (Failurelogs)

df_AlibabaOver_Failurelogs = pd.read_csv('/media/erb/hdd1/DataSet/alibabaOvertime/ssd_failure_label/ssd_failure_label.csv')

In [3]:
# Loading dataset - Alibaba Snapshot (TimeStamps of failed SSDs, SMART attributes in 39 columns, SSDs location, applications, SSD models and Disk ID)

df_AlibabaSnapShot_FailuresAppsLocation = pd.read_csv('/media/erb/hdd1/DataSet/alibabaSnapShot/ssd_failure_tag/ssd_failure_tag.csv')

In [6]:
# Merging Failures and location datasets and fixing columns types (to have ssd failure data that has location, failure time, and smart att)
df_Failurelogs_FailuresAppsLocation =  pd.merge(df_AlibabaOver_Failurelogs, df_AlibabaSnapShot_FailuresAppsLocation, how = 'inner', on = ['disk_id', 'failure_time'])

# Changing failure time column to datetime type
df_Failurelogs_FailuresAppsLocation['failure_time'] =  pd.to_datetime(df_Failurelogs_FailuresAppsLocation['failure_time'])

# Removing duplicates
df_Failurelogs_FailuresAppsLocation = df_Failurelogs_FailuresAppsLocation.drop_duplicates(subset='disk_id', keep="first")

# Forcing sorting
df_Failurelogs_FailuresAppsLocation = df_Failurelogs_FailuresAppsLocation.sort_values(by=['failure_time'], ascending=True)

df_Failurelogs_FailuresAppsLocation.shape
# Changing data type
#df_Failurelogs_FailuresAppsLocation = df_Failurelogs_FailuresAppsLocation.astype(datatype)

ValueError: keep must be either "first", "last" or False

In [ ]:
df_Failurelogs_FailuresAppsLocation = 

dataset_write_cenUm = dataset_cenUm.query('operation == "W" & individual_total == "T"')

dataset_write_cenUm.iops_total[dataset_write_cenUm.iops_total != 0] = (1/dataset_write_cenUm.iops_total)

In [ ]:
fig = sns.catplot(x="capacity", y="iops_total", hue="object_size", data=dataset_write_cenUm,
               row="pattern", col="work_outstd", kind="bar", ci=90, palette="Blues_d", aspect=0.9, height=4.5
                  , legend_out = True, margin_titles = True)

fig.set_axis_labels("Capacity", "IOPS")
fig.set_xticklabels(["80GB HDD", "500GB HDD", "1TB HDD", "1TB WDHDD", "120GB SSD", "Hybrid"])

plt.savefig('write_iops.pdf', dpi=1200)

In [2]:
# Loading AlibabaOvertime dataset using Pandas

start_date = date(2018, 2, 1)
end_date = date(2018, 5, 1)
delta = timedelta(days=1)
df_AlibabaOver_SMARTlogs = pd.DataFrame()

while start_date <= end_date:
    path = Path('/media/erb/hdd1/DataSet/alibabaOvertime/smartlogs/' + start_date.strftime("%Y%m%d") + '.csv')

    if path.is_file(): # checking if a particular file for a specific date is missing
        df_AlibabaOver_SMARTlogs = pd.read_csv(path)
        df_AlibabaOver_SMARTlogs = pd.DataFrame(df_AlibabaOver_SMARTlogs)
        #df_AlibabaOver_SMARTlogs = pd.concat([df_AlibabaOver_SMARTlogs, df_AlibabaOver_SMARTlogsTemp], ignore_index=True)
    start_date += delta

In [ ]:
# Loading AlibabaOvertime dataset using Dask

start_date = date(2018, 2, 1)
start_dateAux = start_date
end_date = date(2018, 5, 31)
delta = timedelta(days=1)

while start_date <= end_date:
    path = Path('/media/erb/hdd1/DataSet/alibabaOvertime/smartlogs/' + start_date.strftime("%Y%m%d") + '.csv')

    if path.is_file(): # checking if a particular file for a specific date is missing
        df_AlibabaOver_SMARTlogsTemp = dd.read_csv(path)
        if start_date == start_dateAux:
            df_AlibabaOver_SMARTlogs = df_AlibabaOver_SMARTlogsTemp
        else: 
            df_AlibabaOver_SMARTlogs = dd.concat([df_AlibabaOver_SMARTlogs, df_AlibabaOver_SMARTlogsTemp])
    start_date += delta

In [35]:
# Testing Alibaba dataset

# settings to display all columns
pd.set_option("display.max_columns", None)

#df_AlibabaOver_SMARTlogs.head()
#df_AlibabaOver_Failurelogs.head(10)
#df_AlibabaSnapShot_FailuresAppsLocation.head()
#df_Failurelogs_FailuresAppsLocation.head(10)
#df_Failurelogs_FailuresAppsLocation.query('model_x == "MA2"')
#df_Failurelogs_FailuresAppsLocation.dtypes





,model_x,failure_time_x,disk_id,model_y,failure_time_y,failure,app,r_5,n_5,r_183,n_183,r_184,n_184,r_187,n_187,r_195,n_195,r_197,n_197,r_199,n_199,r_program,n_program,r_erase,n_erase,n_blocks,n_wearout,r_241,n_241,r_242,n_242,r_9,n_9,r_12,n_12,r_174,n_174,n_175,node_id,rack_id,machine_room_id
0,MA2,2019-03-22 10:24:38,4711,A3,2018-08-11 03:19:22,1,RM,2.0,93.0,NaN,NaN,0.0,100.0,0.0,100.0,2.604913e+09,100.0,0.0,100.0,63.0,100.0,8.0,100.0,0.0,100.0,130.0,1.0,NaN,NaN,NaN,NaN,42675.0,100.0,40.0,100.0,37.0,100.0,100.0,27760,10547,408
1,MA2,2019-03-22 10:24:38,4711,A2,2019-03-22 10:24:38,1,RM,2.0,100.0,0.0,100.0,0.0,100.0,0.0,100.0,NaN,NaN,0.0,100.0,11.0,100.0,4.0,100.0,0.0,100.0,99.0,69.0,8344060.0,100.0,647937.0,100.0,37998.0,100.0,8579.0,100.0,8558.0,100.0,100.0,9780,19604,235
2,MA1,2018-08-11 03:19:22,4711,A3,2018-08-11 03:19:22,1,RM,2.0,93.0,NaN,NaN,0.0,100.0,0.0,100.0,2.604913e+09,100.0,0.0,100.0,63.0,100.0,8.0,100.0,0.0,100.0,130.0,1.0,NaN,NaN,NaN,NaN,42675.0,100.0,40.0,100.0,37.0,100.0,100.0,27760,10547,408
3,MA1,2018-08-11 03:19:22,4711,A2,2019-03-22 10:24:38,1,RM,2.0,100.0,0.0,100.0,0.0,100.0,0.0,100.0,NaN,NaN,0.0,100.0,11.0,100.0,4.0,100.0,0.0,100.0,99.0,69.0,8344060.0,100.0,647937.0,100.0,37998.0,100.0,8579.0,100.0,8558.0,100.0,100.0,9780,19604,235
4,MA2,2019-02-22 04:56:06,82064,A2,2019-02-22 04:56:06,1,SS,0.0,100.0,0.0,100.0,0.0,100.0,0.0,100.0,NaN,NaN,0.0,100.0,0.0,100.0,0.0,100.0,0.0,100.0,100.0,8.0,49839324.0,100.0,30328338.0,100.0,28553.0,100.0,11.0,100.0,8.0,100.0,100.0,160662,16416,212
5,MA2,2018-12-17 12:16:33,32311,A2,2018-12-17 12:16:33,1,none,0.0,100.0,6906.0,100.0,0.0,100.0,0.0,100.0,NaN,NaN,0.0,100.0,15019.0,100.0,0.0,100.0,0.0,100.0,100.0,89.0,5971482.0,100.0,1933774.0,100.0,38380.0,100.0,130.0,100.0,126.0,100.0,100.0,64300,4708,491
6,MA2,2018-05-19 17:32:03,18316,A2,2018-05-19 17:32:03,1,RM,0.0,100.0,0.0,100.0,0.0,100.0,0.0,100.0,NaN,NaN,0.0,100.0,0.0,100.0,0.0,100.0,0.0,100.0,100.0,91.0,4033465.0,100.0,876219.0,100.0,35637.0,100.0,73.0,100.0,71.0,100.0,100.0,36496,10078,455
7,MA2,2018-10-25 04:00:50,32466,A2,2018-10-25 04:00:50,1,RM,0.0,100.0,0.0,100.0,0.0,100.0,0.0,100.0,NaN,NaN,0.0,100.0,0.0,100.0,0.0,100.0,0.0,100.0,100.0,91.0,3251023.0,100.0,320966.0,100.0,34202.0,100.0,99.0,100.0,96.0,100.0,100.0,64549,743,400
8,MA2,2018-12-20 03:44:54,30501,A2,2018-12-20 03:44:54,1,none,5.0,100.0,0.0,100.0,0.0,100.0,0.0,100.0,NaN,NaN,0.0,100.0,0.0,100.0,0.0,100.0,0.0,100.0,99.0,61.0,14677712.0,100.0,3985934.0,100.0,39858.0,100.0,32.0,100.0,28.0,100.0,100.0,60836,28394,667
9,MA2,2019-03-22 10:28:12,76145,A2,2019-03-22 10:28:12,1,RM,0.0,100.0,0.0,100.0,0.0,100.0,0.0,100.0,NaN,NaN,0.0,100.0,205.0,100.0,0.0,100.0,0.0,100.0,100.0,80.0,7563780.0,100.0,40399280.0,100.0,37841.0,100.0,5518.0,100.0,5510.0,100.0,100.0,149063,19604,235


In [ ]:
# Loading Backblaze dataset

start_dateBB = date(2021, 12, 29)
start_dateAuxBB = start_dateBB
end_dateBB = date(2021, 12, 31)
deltaBB = timedelta(days=1)

while start_dateBB <= end_dateBB:
    pathBB = Path('/media/erb/hdd1/DataSet/backblaze/smartlogs/' + start_dateBB.strftime("%Y-%m-%d") + '.csv')

    if pathBB.is_file(): # checking if a particular file for a specific date is missing
        df_BackBlaze_SMARTlogsTemp = pd.read_csv(pathBB)
        if start_dateBB == start_dateAuxBB: # due to dask instancing of variables
            df_BackBlaze_SMARTlogs = df_BackBlaze_SMARTlogsTemp
        else: 
            df_BackBlaze_SMARTlogs = pd.concat([df_BackBlaze_SMARTlogs, df_BackBlaze_SMARTlogsTemp])
    start_dateBB += deltaBB  

In [ ]:
# Testing BackBlaze dataset

df_BackBlaze_SMARTlogs.head()

In [ ]:
#Alterando nome das colunas do arquivo bench e transformando a coluna timestamp no tipo datetime

#PARA HDD 80GB

df_80HDDBenchCenUm4KB.rename(columns = {'Time (s)':'timestamp','Maximum(1) (V)':'maximum_1','Maximum(2) (V)':'maximum_2','Average - Full Screen(1) (V)':'average_1','Average - Full Screen(2) (V)':'average_2'}, inplace=True)
df_80HDDBenchCenUm4KB['timestamp'] =  pd.to_datetime(df_80HDDBenchCenUm4KB['timestamp'])

df_80HDDBenchCenUm128KB.rename(columns = {'Time (s)':'timestamp','Maximum(1) (V)':'maximum_1','Maximum(2) (V)':'maximum_2','Average - Full Screen(1) (V)':'average_1','Average - Full Screen(2) (V)':'average_2'}, inplace=True)
df_80HDDBenchCenUm128KB['timestamp'] =  pd.to_datetime(df_80HDDBenchCenUm128KB['timestamp'])

df_80HDDBenchCenUm512KB.rename(columns = {'Time (s)':'timestamp','Maximum(1) (V)':'maximum_1','Maximum(2) (V)':'maximum_2','Average - Full Screen(1) (V)':'average_1','Average - Full Screen(2) (V)':'average_2'}, inplace=True)
df_80HDDBenchCenUm512KB['timestamp'] =  pd.to_datetime(df_80HDDBenchCenUm512KB['timestamp'])

df_80HDDBenchCenUm1MB.rename(columns = {'Time (s)':'timestamp','Maximum(1) (V)':'maximum_1','Maximum(2) (V)':'maximum_2','Average - Full Screen(1) (V)':'average_1','Average - Full Screen(2) (V)':'average_2'}, inplace=True)
df_80HDDBenchCenUm1MB['timestamp'] =  pd.to_datetime(df_80HDDBenchCenUm1MB['timestamp'])

df_80HDDBenchCenDois4KB.rename(columns = {'Time (s)':'timestamp','Maximum(1) (V)':'maximum_1','Maximum(2) (V)':'maximum_2','Average - Full Screen(1) (V)':'average_1','Average - Full Screen(2) (V)':'average_2'}, inplace=True)
df_80HDDBenchCenDois4KB['timestamp'] =  pd.to_datetime(df_80HDDBenchCenDois4KB['timestamp'])

df_80HDDBenchCenDois128KB.rename(columns = {'Time (s)':'timestamp','Maximum(1) (V)':'maximum_1','Maximum(2) (V)':'maximum_2','Average - Full Screen(1) (V)':'average_1','Average - Full Screen(2) (V)':'average_2'}, inplace=True)
df_80HDDBenchCenDois128KB['timestamp'] =  pd.to_datetime(df_80HDDBenchCenDois128KB['timestamp'])

df_80HDDBenchCenDois512KB.rename(columns = {'Time (s)':'timestamp','Maximum(1) (V)':'maximum_1','Maximum(2) (V)':'maximum_2','Average - Full Screen(1) (V)':'average_1','Average - Full Screen(2) (V)':'average_2'}, inplace=True)
df_80HDDBenchCenDois512KB['timestamp'] =  pd.to_datetime(df_80HDDBenchCenDois512KB['timestamp'])

df_80HDDBenchCenDois1MB.rename(columns = {'Time (s)':'timestamp','Maximum(1) (V)':'maximum_1','Maximum(2) (V)':'maximum_2','Average - Full Screen(1) (V)':'average_1','Average - Full Screen(2) (V)':'average_2'}, inplace=True)
df_80HDDBenchCenDois1MB['timestamp'] =  pd.to_datetime(df_80HDDBenchCenDois1MB['timestamp'])



In [ ]:
# Combinando todos os DataFrames do IOMeter

#Cenario1

df_IOMeterCenUm_Total = [dfTemp_80HDDMeterCenUm4KB,dfTemp_80HDDMeterCenUm128KB,dfTemp_80HDDMeterCenUm512KB,dfTemp_80HDDMeterCenUm1MB,dfTemp_500HDDMeterCenUm4KB,dfTemp_500HDDMeterCenUm128KB,dfTemp_500HDDMeterCenUm512KB,dfTemp_500HDDMeterCenUm1MB
                         ,dfTemp_1HDDMeterCenUm4KB,dfTemp_1HDDMeterCenUm128KB,dfTemp_1HDDMeterCenUm512KB,dfTemp_1HDDMeterCenUm1MB,dfTemp_1WDHDDMeterCenUm4KB,dfTemp_1WDHDDMeterCenUm128KB,dfTemp_1WDHDDMeterCenUm512KB,dfTemp_1WDHDDMeterCenUm1MB,
                         dfTemp_120SSDMeterCenUm4KB,dfTemp_120SSDMeterCenUm128KB,dfTemp_120SSDMeterCenUm512KB,dfTemp_120SSDMeterCenUm1MB,dfTemp_HybridMeterCenUm4KB,dfTemp_HybridMeterCenUm128KB,dfTemp_HybridMeterCenUm512KB,dfTemp_HybridMeterCenUm1MB]

#Cenario2

df_IOMeterCenDois_Total = [dfTemp_80HDDMeterCenDois4KB,dfTemp_80HDDMeterCenDois128KB,dfTemp_80HDDMeterCenDois512KB,dfTemp_80HDDMeterCenDois1MB,dfTemp_500HDDMeterCenDois4KB,dfTemp_500HDDMeterCenDois128KB,dfTemp_500HDDMeterCenDois512KB,dfTemp_500HDDMeterCenDois1MB
                         ,dfTemp_1HDDMeterCenDois4KB,dfTemp_1HDDMeterCenDois128KB,dfTemp_1HDDMeterCenDois512KB,dfTemp_1HDDMeterCenDois1MB,dfTemp_1WDHDDMeterCenDois4KB,dfTemp_1WDHDDMeterCenDois128KB,dfTemp_1WDHDDMeterCenDois512KB,dfTemp_1WDHDDMeterCenDois1MB
                           ,dfTemp_120SSDMeterCenDois4KB,dfTemp_120SSDMeterCenDois128KB,dfTemp_120SSDMeterCenDois512KB,dfTemp_120SSDMeterCenDois1MB,dfTemp_HybridMeterCenDois4KB,dfTemp_HybridMeterCenDois128KB,dfTemp_HybridMeterCenDois512KB,dfTemp_HybridMeterCenDois1MB]


df_IOMeter_Total = df_IOMeterCenUm_Total + df_IOMeterCenDois_Total




In [ ]:
with open('dataset.csv', 'w', newline='') as f:
    dataset = csv.DictWriter(f, fieldnames = ['timestamp','capacity','object_size','pattern','operation','scenario','work_outstd','repetition','sample_count','individual_total','iops_total','iops_read','iops_write','avg_response_time_total','avg_response_time_read','avg_response_time_write','maximum_response_time_total','maximum_response_time_read','maximum_response_time_write','errors_total','errors_read','errors_write','cpu_utilization','power_maximum1_hdd','power_maximum2_hdd','power_average1_hdd','power_average2_hdd','power_maximum1_ssd','power_maximum2_ssd','power_average1_ssd','power_average2_ssd', 'power_total'], delimiter = ',')
    dataset.writeheader()
    #dataset= csv.writer(f, delimiter=',')
    #dataset.writerow(['time_stamp','capacity','object_size','pattern','operation','scenario','work_outstd','repetition','individual_total','iops_total','iops_read','iops_write','avg_response_time_total','avg_response_time_read','avg_response_time_write','maximum_response_time_total','maximum_response_time_read','maximum_response_time_write','errors_total','errors_read','errors_write','cpu_utilization','power'])
    #arquivosaida.writerow({1, 2}) -- inserindo dados
    
    row_index=0
    workers_outstanding=1
    encontrou = False
    sample_count=0
    repetition=1
    pattern=1
    kind='hdd'
    aux_voltage_current= [0,0,0,0]
    aux_voltage_total= [0,0,0,0]
    aux_voltage_current_series = [0,0,0,0]
    aux_voltage_current_hybrid= [0,0,0,0]
    aux_voltage_total_hybrid= [0,0,0,0]
    aux_voltage_current_series_hybrid = [0,0,0,0]
    cenario_atual = 1
    df_IOMeter_index = 0
    
    #indiv_total='I'
    for df_IOMeter in df_IOMeter_Total:                  
        
        if (df_IOMeter_index<4 or (df_IOMeter_index>23 and df_IOMeter_index<28)):
            capacity = '80GBHDD'
            kind='hdd'
            if ((df_IOMeter_index == 0) or (df_IOMeter_index==24)):
                object_size = '4KB'
            elif ((df_IOMeter_index == 1) or (df_IOMeter_index==25)):
                object_size = '128KB'
            elif ((df_IOMeter_index == 2) or (df_IOMeter_index==26)):
                object_size = '512KB'
            elif ((df_IOMeter_index == 3) or (df_IOMeter_index==27)):
                object_size = '1MB'                
        elif ((df_IOMeter_index > 3 and df_IOMeter_index < 8) or (df_IOMeter_index>27 and df_IOMeter_index<32)):
            capacity = '500GBHDD'
            kind='hdd'
            if ((df_IOMeter_index == 4) or (df_IOMeter_index==28)):
                object_size = '4KB'
            elif ((df_IOMeter_index == 5) or (df_IOMeter_index==29)):
                object_size = '128KB'
            elif ((df_IOMeter_index == 6) or (df_IOMeter_index==30)):
                object_size = '512KB'
            elif ((df_IOMeter_index == 7) or (df_IOMeter_index==31)):
                object_size = '1MB'
        elif ((df_IOMeter_index > 7 and df_IOMeter_index < 12) or (df_IOMeter_index>31 and df_IOMeter_index<36)):
            capacity = '1TBHDD'
            kind='hdd'
            if ((df_IOMeter_index == 8) or (df_IOMeter_index==32)):
                object_size = '4KB'
            elif ((df_IOMeter_index == 9) or (df_IOMeter_index==33)):
                object_size = '128KB'
            elif ((df_IOMeter_index == 10) or (df_IOMeter_index==34)):
                object_size = '512KB'
            elif ((df_IOMeter_index == 11) or (df_IOMeter_index==35)):
                object_size = '1MB'
        elif ((df_IOMeter_index > 11 and df_IOMeter_index<16) or (df_IOMeter_index>35 and df_IOMeter_index<40)):
            capacity = '1TBWDHDD'
            kind='hdd'
            if ((df_IOMeter_index == 12) or (df_IOMeter_index==36)):
                object_size = '4KB'
            elif ((df_IOMeter_index == 13) or (df_IOMeter_index==37)):
                object_size = '128KB'
            elif ((df_IOMeter_index == 14) or (df_IOMeter_index==38)):
                object_size = '512KB'
            elif ((df_IOMeter_index == 15) or (df_IOMeter_index==39)):
                object_size = '1MB'
        elif ((df_IOMeter_index > 15 and df_IOMeter_index<20) or (df_IOMeter_index>39 and df_IOMeter_index<44)):
            capacity = '120GBSSD'
            kind = 'ssd'
            if ((df_IOMeter_index == 16) or (df_IOMeter_index==40)):
                object_size = '4KB'
            elif ((df_IOMeter_index == 17) or (df_IOMeter_index==41)):
                object_size = '128KB'
            elif ((df_IOMeter_index == 18) or (df_IOMeter_index==42)):
                object_size = '512KB'
            elif ((df_IOMeter_index == 19) or (df_IOMeter_index==43)):
                object_size = '1MB'
        elif ((df_IOMeter_index > 19 and df_IOMeter_index<24) or (df_IOMeter_index>43 and df_IOMeter_index<48)):
            capacity = 'Hybrid'
            kind = 'hybrid'
            if ((df_IOMeter_index == 20) or (df_IOMeter_index==44)):
                object_size = '4KB'
            elif ((df_IOMeter_index == 21) or (df_IOMeter_index==45)):
                object_size = '128KB'
            elif ((df_IOMeter_index == 22) or (df_IOMeter_index==46)):
                object_size = '512KB'
            elif ((df_IOMeter_index == 23) or (df_IOMeter_index==47)):
                object_size = '1MB'
        for rows in df_IOMeter: 
            if (encontrou):
                #print (rows[0])
                aux_date = datetime.strptime(rows[0], '%Y-%m-%d %H:%M:%S:%f')
                if (workers_outstanding == 1):
                    sample_count = sample_count + 1
                    if (rows[1] == 'ALL'):
                        if (pattern == 1):
                            if (kind == 'hdd'):
                                dataset.writerow({'timestamp': aux_date,'capacity': capacity,'object_size':object_size,'pattern':'R','operation':'W','scenario': cenario_atual,'work_outstd': workers_outstanding,'repetition': repetition,'sample_count': sample_count,'individual_total': 'T','iops_total': rows[7],'iops_read':rows[8],'iops_write':rows[9],'avg_response_time_total':rows[18],'avg_response_time_read':rows[19],'avg_response_time_write':rows[20],'maximum_response_time_total':rows[23],'maximum_response_time_read':rows[24],'maximum_response_time_write':rows[25],'errors_total':rows[28],'errors_read':rows[29],'errors_write':rows[30],'cpu_utilization':rows[49],'power_maximum1_hdd':(aux_voltage_total[0]/(sample_count-1)),'power_maximum2_hdd':(aux_voltage_total[1]/(sample_count-1)),'power_average1_hdd':(aux_voltage_total[2]/(sample_count-1)),'power_average2_hdd':(aux_voltage_total[3]/(sample_count-1))})
                            elif (kind == 'ssd'):
                                dataset.writerow({'timestamp': aux_date,'capacity': capacity,'object_size':object_size,'pattern':'R','operation':'W','scenario': cenario_atual,'work_outstd': workers_outstanding,'repetition': repetition,'sample_count': sample_count,'individual_total': 'T','iops_total': rows[7],'iops_read':rows[8],'iops_write':rows[9],'avg_response_time_total':rows[18],'avg_response_time_read':rows[19],'avg_response_time_write':rows[20],'maximum_response_time_total':rows[23],'maximum_response_time_read':rows[24],'maximum_response_time_write':rows[25],'errors_total':rows[28],'errors_read':rows[29],'errors_write':rows[30],'cpu_utilization':rows[49],'power_maximum1_ssd':(aux_voltage_total[0]/(sample_count-1)),'power_maximum2_ssd':(aux_voltage_total[1]/(sample_count-1)),'power_average1_ssd':(aux_voltage_total[2]/(sample_count-1)),'power_average2_ssd':(aux_voltage_total[3]/(sample_count-1))})
                            else:
                                dataset.writerow({'timestamp': aux_date,'capacity': capacity,'object_size':object_size,'pattern':'R','operation':'W','scenario': cenario_atual,'work_outstd': workers_outstanding,'repetition': repetition,'sample_count': sample_count,'individual_total': 'T','iops_total': rows[7],'iops_read':rows[8],'iops_write':rows[9],'avg_response_time_total':rows[18],'avg_response_time_read':rows[19],'avg_response_time_write':rows[20],'maximum_response_time_total':rows[23],'maximum_response_time_read':rows[24],'maximum_response_time_write':rows[25],'errors_total':rows[28],'errors_read':rows[29],'errors_write':rows[30],'cpu_utilization':rows[49],'power_maximum1_ssd':(aux_voltage_total[0]/(sample_count-1)),'power_maximum2_ssd':(aux_voltage_total[1]/(sample_count-1)),'power_average1_ssd':(aux_voltage_total[2]/(sample_count-1)),'power_average2_ssd':(aux_voltage_total[3]/(sample_count-1)),'power_maximum1_hdd':(aux_voltage_total_hybrid[0]/(sample_count-1)),'power_maximum2_hdd':(aux_voltage_total_hybrid[1]/(sample_count-1)),'power_average1_hdd':(aux_voltage_total_hybrid[2]/(sample_count-1)),'power_average2_hdd':(aux_voltage_total_hybrid[3]/(sample_count-1))})
  

In [ ]:
# Carregando dados filtrados em um dataframe para graficos de performance

dataset = pd.read_csv("dataset.csv")

# Separando por cenários

##Cenário 1
dataset_cenUm = dataset.query('scenario == 1')

##Cenário 2
dataset_cenDois = dataset.query('scenario == 2')

# Separando por tipo de operação

##Cenário 1

### Escrita
dataset_write_cenUm = dataset_cenUm.query('operation == "W" & individual_total == "T"')



In [ ]:
### QUANTIDADE DE CONJUNTOS DE DADOS DO CENARIO 1

dataset_cenUm_total.shape

In [ ]:
# INVERTENDO COLUNA IOPS TOTAL para performance

## Cenario 1

dataset_write_cenUm.iops_total[dataset_write_cenUm.iops_total != 0] = (1/dataset_write_cenUm.iops_total)

dataset_read_cenUm.iops_total[dataset_read_cenUm.iops_total != 0] = (1/dataset_read_cenUm.iops_total)

dataset_mix_cenUm.iops_total[dataset_mix_cenUm.iops_total != 0] = (1/dataset_mix_cenUm.iops_total)



In [ ]:
## RETIRANDO RUÍDOS DAS MEDIÇÕES DE TENSÃO para performance

##Cenário 1

### Escrita

dataset_write_cenUm.power_average1_hdd = dataset_write_cenUm.power_average1_hdd.astype(str)
dataset_write_cenUm['power_average1_hdd'] = dataset_write_cenUm['power_average1_hdd'].apply(lambda x: x.split("e")[0])
dataset_write_cenUm.power_average1_hdd = dataset_write_cenUm.power_average1_hdd.astype(float)

dataset_write_cenUm.power_average2_hdd = dataset_write_cenUm.power_average2_hdd.astype(str)
dataset_write_cenUm['power_average2_hdd'] = dataset_write_cenUm['power_average2_hdd'].apply(lambda x: x.split("e")[0])
dataset_write_cenUm.power_average2_hdd = dataset_write_cenUm.power_average2_hdd.astype(float)

dataset_write_cenUm.power_average1_ssd = dataset_write_cenUm.power_average1_ssd.astype(str)
dataset_write_cenUm['power_average1_ssd'] = dataset_write_cenUm['power_average1_ssd'].apply(lambda x: x.split("e")[0])
dataset_write_cenUm.power_average1_ssd = dataset_write_cenUm.power_average1_ssd.astype(float)

dataset_write_cenUm.power_average2_ssd = dataset_write_cenUm.power_average2_ssd.astype(str)
dataset_write_cenUm['power_average2_ssd'] = dataset_write_cenUm['power_average2_ssd'].apply(lambda x: x.split("e")[0])
dataset_write_cenUm.power_average2_ssd = dataset_write_cenUm.power_average2_ssd.astype(float)

### Leitura

dataset_read_cenUm.power_average1_hdd = dataset_read_cenUm.power_average1_hdd.astype(str)
dataset_read_cenUm['power_average1_hdd'] = dataset_read_cenUm['power_average1_hdd'].apply(lambda x: x.split("e")[0])
dataset_read_cenUm.power_average1_hdd = dataset_read_cenUm.power_average1_hdd.astype(float)

dataset_read_cenUm.power_average2_hdd = dataset_read_cenUm.power_average2_hdd.astype(str)
dataset_read_cenUm['power_average2_hdd'] = dataset_read_cenUm['power_average2_hdd'].apply(lambda x: x.split("e")[0])
dataset_read_cenUm.power_average2_hdd = dataset_read_cenUm.power_average2_hdd.astype(float)

dataset_read_cenUm.power_average1_ssd = dataset_read_cenUm.power_average1_ssd.astype(str)
dataset_read_cenUm['power_average1_ssd'] = dataset_read_cenUm['power_average1_ssd'].apply(lambda x: x.split("e")[0])
dataset_read_cenUm.power_average1_ssd = dataset_read_cenUm.power_average1_ssd.astype(float)

dataset_read_cenUm.power_average2_ssd = dataset_read_cenUm.power_average2_ssd.astype(str)
dataset_read_cenUm['power_average2_ssd'] = dataset_read_cenUm['power_average2_ssd'].apply(lambda x: x.split("e")[0])
dataset_read_cenUm.power_average2_ssd = dataset_read_cenUm.power_average2_ssd.astype(float)

### Mix

dataset_mix_cenUm.power_average1_hdd = dataset_mix_cenUm.power_average1_hdd.astype(str)
dataset_mix_cenUm['power_average1_hdd'] = dataset_mix_cenUm['power_average1_hdd'].apply(lambda x: x.split("e")[0])
dataset_mix_cenUm.power_average1_hdd = dataset_mix_cenUm.power_average1_hdd.astype(float)

dataset_mix_cenUm.power_average2_hdd = dataset_mix_cenUm.power_average2_hdd.astype(str)
dataset_mix_cenUm['power_average2_hdd'] = dataset_mix_cenUm['power_average2_hdd'].apply(lambda x: x.split("e")[0])
dataset_mix_cenUm.power_average2_hdd = dataset_mix_cenUm.power_average2_hdd.astype(float)

dataset_mix_cenUm.power_average1_ssd = dataset_mix_cenUm.power_average1_ssd.astype(str)
dataset_mix_cenUm['power_average1_ssd'] = dataset_mix_cenUm['power_average1_ssd'].apply(lambda x: x.split("e")[0])
dataset_mix_cenUm.power_average1_ssd = dataset_mix_cenUm.power_average1_ssd.astype(float)

dataset_mix_cenUm.power_average2_ssd = dataset_mix_cenUm.power_average2_ssd.astype(str)
dataset_mix_cenUm['power_average2_ssd'] = dataset_mix_cenUm['power_average2_ssd'].apply(lambda x: x.split("e")[0])
dataset_mix_cenUm.power_average2_ssd = dataset_mix_cenUm.power_average2_ssd.astype(float)

##Cenário 2

### Escrita

dataset_write_cenDois.power_average1_hdd = dataset_write_cenDois.power_average1_hdd.astype(str)
dataset_write_cenDois['power_average1_hdd'] = dataset_write_cenDois['power_average1_hdd'].apply(lambda x: x.split("e")[0])
dataset_write_cenDois.power_average1_hdd = dataset_write_cenDois.power_average1_hdd.astype(float)

dataset_write_cenDois.power_average2_hdd = dataset_write_cenDois.power_average2_hdd.astype(str)
dataset_write_cenDois['power_average2_hdd'] = dataset_write_cenDois['power_average2_hdd'].apply(lambda x: x.split("e")[0])
dataset_write_cenDois.power_average2_hdd = dataset_write_cenDois.power_average2_hdd.astype(float)

dataset_write_cenDois.power_average1_ssd = dataset_write_cenDois.power_average1_ssd.astype(str)
dataset_write_cenDois['power_average1_ssd'] = dataset_write_cenDois['power_average1_ssd'].apply(lambda x: x.split("e")[0])
dataset_write_cenDois.power_average1_ssd = dataset_write_cenDois.power_average1_ssd.astype(float)

dataset_write_cenDois.power_average2_ssd = dataset_write_cenDois.power_average2_ssd.astype(str)
dataset_write_cenDois['power_average2_ssd'] = dataset_write_cenDois['power_average2_ssd'].apply(lambda x: x.split("e")[0])
dataset_write_cenDois.power_average2_ssd = dataset_write_cenDois.power_average2_ssd.astype(float)

###Leitura

dataset_read_cenDois.power_average1_hdd = dataset_read_cenDois.power_average1_hdd.astype(str)
dataset_read_cenDois['power_average1_hdd'] = dataset_read_cenDois['power_average1_hdd'].apply(lambda x: x.split("e")[0])
dataset_read_cenDois.power_average1_hdd = dataset_read_cenDois.power_average1_hdd.astype(float)

dataset_read_cenDois.power_average2_hdd = dataset_read_cenDois.power_average2_hdd.astype(str)
dataset_read_cenDois['power_average2_hdd'] = dataset_read_cenDois['power_average2_hdd'].apply(lambda x: x.split("e")[0])
dataset_read_cenDois.power_average2_hdd = dataset_read_cenDois.power_average2_hdd.astype(float)

dataset_read_cenDois.power_average1_ssd = dataset_read_cenDois.power_average1_ssd.astype(str)
dataset_read_cenDois['power_average1_ssd'] = dataset_read_cenDois['power_average1_ssd'].apply(lambda x: x.split("e")[0])
dataset_read_cenDois.power_average1_ssd = dataset_read_cenDois.power_average1_ssd.astype(float)

dataset_read_cenDois.power_average2_ssd = dataset_read_cenDois.power_average2_ssd.astype(str)
dataset_read_cenDois['power_average2_ssd'] = dataset_read_cenDois['power_average2_ssd'].apply(lambda x: x.split("e")[0])
dataset_read_cenDois.power_average2_ssd = dataset_read_cenDois.power_average2_ssd.astype(float)

###Mix

dataset_mix_cenDois.power_average1_hdd = dataset_mix_cenDois.power_average1_hdd.astype(str)
dataset_mix_cenDois['power_average1_hdd'] = dataset_mix_cenDois['power_average1_hdd'].apply(lambda x: x.split("e")[0])
dataset_mix_cenDois.power_average1_hdd = dataset_mix_cenDois.power_average1_hdd.astype(float)

dataset_mix_cenDois.power_average2_hdd = dataset_mix_cenDois.power_average2_hdd.astype(str)
dataset_mix_cenDois['power_average2_hdd'] = dataset_mix_cenDois['power_average2_hdd'].apply(lambda x: x.split("e")[0])
dataset_mix_cenDois.power_average2_hdd = dataset_mix_cenDois.power_average2_hdd.astype(float)

dataset_mix_cenDois.power_average1_ssd = dataset_mix_cenDois.power_average1_ssd.astype(str)
dataset_mix_cenDois['power_average1_ssd'] = dataset_mix_cenDois['power_average1_ssd'].apply(lambda x: x.split("e")[0])
dataset_mix_cenDois.power_average1_ssd = dataset_mix_cenDois.power_average1_ssd.astype(float)

dataset_mix_cenDois.power_average2_ssd = dataset_mix_cenDois.power_average2_ssd.astype(str)
dataset_mix_cenDois['power_average2_ssd'] = dataset_mix_cenDois['power_average2_ssd'].apply(lambda x: x.split("e")[0])
dataset_mix_cenDois.power_average2_ssd = dataset_mix_cenDois.power_average2_ssd.astype(float)

In [ ]:
## RETIRANDO RUÍDOS DAS MEDIÇÕES DE TENSÃO para SPN

##Cenário 1

### Escrita

dataset_write_cenUm_SPN.power_average1_hdd = dataset_write_cenUm_SPN.power_average1_hdd.astype(str)
dataset_write_cenUm_SPN['power_average1_hdd'] = dataset_write_cenUm_SPN['power_average1_hdd'].apply(lambda x: x.split("e")[0])
dataset_write_cenUm_SPN.power_average1_hdd = dataset_write_cenUm_SPN.power_average1_hdd.astype(float)



In [ ]:
#dataset_write_cenUm.power_average1_hdd
#dataset_teste = dataset_write_cenUm.query('capacity == "120GBSSD"')
#dataset_teste.power_average2_ssd
dataset_teste = dataset_write_cenDois.query('capacity == "80GBHDD"')
#dataset_teste.power_average1_hdd
dataset_teste.head()

In [ ]:
## PARA PERFORMANCE
## Calculando consumo de energia total hdd 80GB

dataset_write_cenUm.power_total[dataset_write_cenUm.capacity == '80GBHDD'] = (((6.8-dataset_write_cenUm.power_average1_hdd[dataset_write_cenUm.capacity == '80GBHDD'])/(1.08))*(dataset_write_cenUm.power_average1_hdd[dataset_write_cenUm.capacity == '80GBHDD']))+(((12.7-dataset_write_cenUm.power_average2_hdd[dataset_write_cenUm.capacity == '80GBHDD']) /(0.68))*(dataset_write_cenUm.power_average1_hdd[dataset_write_cenUm.capacity == '80GBHDD']))
dataset_read_cenUm.power_total[dataset_read_cenUm.capacity == '80GBHDD'] = (((6.8-dataset_read_cenUm.power_average1_hdd[dataset_read_cenUm.capacity == '80GBHDD']) /(1.08))*(dataset_read_cenUm.power_average1_hdd[dataset_read_cenUm.capacity == '80GBHDD']))+(((12.7-dataset_read_cenUm.power_average2_hdd[dataset_read_cenUm.capacity == '80GBHDD']) /(0.68))*(dataset_read_cenUm.power_average2_hdd[dataset_read_cenUm.capacity == '80GBHDD']))
dataset_mix_cenUm.power_total[dataset_mix_cenUm.capacity == '80GBHDD'] = (((6.8-dataset_mix_cenUm.power_average1_hdd[dataset_mix_cenUm.capacity == '80GBHDD']) /(1.08))*(dataset_mix_cenUm.power_average1_hdd[dataset_mix_cenUm.capacity == '80GBHDD']))+(((12.7-dataset_mix_cenUm.power_average2_hdd[dataset_mix_cenUm.capacity == '80GBHDD']) /(0.68))*(dataset_mix_cenUm.power_average2_hdd[dataset_mix_cenUm.capacity == '80GBHDD']))

dataset_write_cenDois.power_total[dataset_write_cenDois.capacity == '80GBHDD'] = (((6.8-dataset_write_cenDois.power_average1_hdd[dataset_write_cenDois.capacity == '80GBHDD']) /(1.08))*(dataset_write_cenDois.power_average1_hdd[dataset_write_cenDois.capacity == '80GBHDD']))+(((12.7-dataset_write_cenDois.power_average2_hdd[dataset_write_cenDois.capacity == '80GBHDD']) /(0.68))*(dataset_write_cenDois.power_average2_hdd[dataset_write_cenDois.capacity == '80GBHDD']))
dataset_read_cenDois.power_total[dataset_read_cenDois.capacity == '80GBHDD'] = (((6.8-dataset_read_cenDois.power_average1_hdd[dataset_read_cenDois.capacity == '80GBHDD'])/(1.08))*(dataset_read_cenDois.power_average1_hdd[dataset_read_cenDois.capacity == '80GBHDD']))+(((12.7-dataset_read_cenDois.power_average2_hdd[dataset_read_cenDois.capacity == '80GBHDD'])/(0.68))*(dataset_read_cenDois.power_average2_hdd[dataset_read_cenDois.capacity == '80GBHDD']))
dataset_mix_cenDois.power_total[dataset_mix_cenDois.capacity == '80GBHDD'] = (((6.8-dataset_mix_cenDois.power_average1_hdd[dataset_mix_cenDois.capacity == '80GBHDD'])/(1.08))*(dataset_mix_cenDois.power_average1_hdd[dataset_mix_cenDois.capacity == '80GBHDD']))+(((12.7-dataset_mix_cenDois.power_average2_hdd[dataset_mix_cenDois.capacity == '80GBHDD'])/(0.68))*(dataset_mix_cenDois.power_average2_hdd[dataset_mix_cenDois.capacity == '80GBHDD']))




In [ ]:
dataset_teste = dataset_write_cenDois.query('capacity == "1TBWDHDD"')
dataset_teste.power_total

In [ ]:
# APLICANDO O MODULO para performance

## hdd 80GB

dataset_write_cenUm.power_total[dataset_write_cenUm.capacity == '80GBHDD'] = abs(dataset_write_cenUm.power_total[dataset_write_cenUm.capacity == '80GBHDD'])
dataset_read_cenUm.power_total[dataset_read_cenUm.capacity == '80GBHDD'] = abs(dataset_read_cenUm.power_total[dataset_read_cenUm.capacity == '80GBHDD'])
dataset_mix_cenUm.power_total[dataset_mix_cenUm.capacity == '80GBHDD'] = abs(dataset_mix_cenUm.power_total[dataset_mix_cenUm.capacity == '80GBHDD'])



In [ ]:
## Salvando valores de energia consumida, no csv, para adicionar na analise individual do storage hibrido
# Para Performance

dataset_write_cenUm_hdd = dataset_write_cenUm.query('capacity == "Hybrid"')
dataset_read_cenUm_hdd = dataset_read_cenUm.query('capacity == "Hybrid"')
dataset_mix_cenUm_hdd = dataset_mix_cenUm.query('capacity == "Hybrid"')




dataset_write_cenUm_hdd.power_total[dataset_write_cenUm_hdd.capacity == 'Hybrid'] = abs((((12.79-dataset_write_cenUm_hdd.power_average1_hdd[dataset_write_cenUm_hdd.capacity == 'Hybrid'])/(0.52))*(dataset_write_cenUm_hdd.power_average1_hdd[dataset_write_cenUm_hdd.capacity == 'Hybrid']))+(((6.78-dataset_write_cenUm_hdd.power_average2_hdd[dataset_write_cenUm_hdd.capacity == 'Hybrid'])/(0.43))*(dataset_write_cenUm_hdd.power_average2_hdd[dataset_write_cenUm_hdd.capacity == 'Hybrid'])))
dataset_read_cenUm_hdd.power_total[dataset_read_cenUm_hdd.capacity == 'Hybrid'] = abs((((12.79-dataset_read_cenUm_hdd.power_average1_hdd[dataset_read_cenUm_hdd.capacity == 'Hybrid'])/(0.52))*(dataset_read_cenUm_hdd.power_average1_hdd[dataset_read_cenUm_hdd.capacity == 'Hybrid']))+(((6.78-dataset_read_cenUm_hdd.power_average2_hdd[dataset_read_cenUm_hdd.capacity == 'Hybrid'])/(0.43))*(dataset_read_cenUm_hdd.power_average2_hdd[dataset_read_cenUm_hdd.capacity == 'Hybrid'])))
dataset_mix_cenUm_hdd.power_total[dataset_mix_cenUm_hdd.capacity == 'Hybrid'] = abs((((12.79-dataset_mix_cenUm_hdd.power_average1_hdd[dataset_mix_cenUm_hdd.capacity == 'Hybrid'])/(0.52))*(dataset_mix_cenUm_hdd.power_average1_hdd[dataset_mix_cenUm_hdd.capacity == 'Hybrid']))+(((6.78-dataset_mix_cenUm_hdd.power_average2_hdd[dataset_mix_cenUm_hdd.capacity == 'Hybrid'])/(0.43))*(dataset_mix_cenUm_hdd.power_average2_hdd[dataset_mix_cenUm_hdd.capacity == 'Hybrid'])))



In [ ]:
## FILTRO DE DADOS PARA Performance PARA DOE Minitab (com 20 replicates) - cenario 2

scenario = '2'
storages = ['80GBHDD', '500GBHDD', '1TBHDD', '1TBWDHDD', '120GBSSD']
metrics = ['iops_total', 'avg_response_time_total', 'power_total', 'cpu_utilization']
objects_size = ['4KB', '128KB', '512KB', '1MB']
patterns = ['80R', 'R', 'S']
replicates = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20']
work_outstd = ['1', '4', '16','64']
operations = ['Write', 'Read', 'Mix']
dataset_metric2 = pd.DataFrame()

for metric in metrics:
    for run in replicates:
        for operation in operations:
             for storage in storages:
                    for object_size in objects_size:
                        for pattern in patterns:
                            for work in work_outstd:

                                if operation == 'Write':
                                    dataset_filter = dataset_write_cenDois.query('repetition == @run & scenario == @scenario & capacity == @storage & object_size == @object_size & pattern == @pattern & work_outstd == @work')             
                                elif operation == 'Read':
                                    dataset_filter = dataset_read_cenDois.query('repetition == @run & scenario == @scenario & capacity == @storage & object_size == @object_size & pattern == @pattern & work_outstd == @work')
                                else:
                                    dataset_filter = dataset_mix_cenDois.query('repetition == @run & scenario == @scenario & capacity == @storage & object_size == @object_size & pattern == @pattern & work_outstd == @work')


                                Columns = [metric]
                                dataset_metric = pd.DataFrame(columns=Columns)
                                dataset_metric[metric] = dataset_filter[metric]
                                dataset_metric2 = dataset_metric2.append(dataset_metric)
                                #dataset_metric['repetition'] = dataset_filter.repetition
    filename = "Performance/" + "DOE/" + scenario + "/" + metric + "/" + "FilteredData.csv"
    dataset_metric2.to_csv(filename, index=False)
    dataset_metric2 = pd.DataFrame()

In [ ]:
## FILTRO DE DADOS PARA O MODELO SPN 1 (easyfit)

storages = ['80GBHDD', '500GBHDD', '1TBHDD', '1TBWDHDD', '120GBSSD', 'Hybrid']
metrics = ['iops_total', 'avg_response_time_total', 'power_total']
objects_size = ['4KB', '128KB', '512KB', '1MB']
patterns = ['80R', 'R', 'S']
work_outstd = ['1', '2', '4']
operations = ['Write', 'Read', 'Mix']

for operation in operations:
    for metric in metrics:
        for storage in storages:
            for object_size in objects_size:
                for pattern in patterns:
                    for work in work_outstd:
                        if operation == 'Write':
                            dataset_filter = dataset_write_cenUm_SPN.query('capacity == @storage & object_size == @object_size & pattern == @pattern & work_outstd == @work')             
                        elif operation == 'Read':
                            dataset_filter = dataset_read_cenUm_SPN.query('capacity == @storage & object_size == @object_size & pattern == @pattern & work_outstd == @work')
                        else:
                            dataset_filter = dataset_mix_cenUm_SPN.query('capacity == @storage & object_size == @object_size & pattern == @pattern & work_outstd == @work')
                        Columns = [metric,'repetition']
                        Columns2 = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20']
                        dataset_metric = pd.DataFrame(columns=Columns)
                        dataset_metric2 = pd.DataFrame()
                        dataset_metric[metric] = dataset_filter[metric]
                        dataset_metric['repetition'] = dataset_filter.repetition
                        for col in Columns2:
                            dataset_metric_temp = dataset_metric.query('repetition == @col')
                            dataset_metric3 = pd.DataFrame()
                            dataset_metric3[col] = dataset_metric_temp[metric].get_values()[:]
                            dataset_metric2 = pd.concat([dataset_metric2,dataset_metric3], ignore_index=True, axis=1)
                        filename = "SPN1/" + operation + "/" + metric + "/" + storage + "/" + object_size + "/" + pattern + "/" + work + "/" + "FilteredData.csv"
                        dataset_metric2.to_csv(filename, index=False)

In [ ]:
# CENARIO 1

## WRITE IOPS

fig = sns.catplot(x="capacity", y="iops_total", hue="object_size", data=dataset_write_cenUm,
               row="pattern", col="work_outstd", kind="bar", ci=90, palette="Blues_d", aspect=0.9, height=4.5
                  , legend_out = True, margin_titles = True)

fig.set_axis_labels("Capacity", "IOPS")
fig.set_xticklabels(["80GB HDD", "500GB HDD", "1TB HDD", "1TB WDHDD", "120GB SSD", "Hybrid"])

plt.savefig('write_iops.pdf', dpi=1200)

In [ ]:
teste = dataset_write_cenUm.query('pattern == "R" & work_outstd == 2 & capacity == "1TBHDD" & object_size == "1MB"')
teste.power_average1

teste.power_average1 = teste.power_average1.astype(str)
teste['power_average1'] = teste['power_average1'].apply(lambda x: x.split("e")[0])
teste.power_average1 = teste.power_average1.astype(float)
teste.power_average1


## EXAMPLES

#teste.power_average1.mean()
#dataset_write_cenUm.iops_total.isnull().sum()
#teste.power_average1[teste['power_average1'].str.contains('e')] = teste['power_average1'].apply(lambda x: x.split("e")[0])
#df['coluna'] = df['coluna'].apply(lambda x: x.split("%")[1])
#df.column = df.column.astype(float)
#dataset_write_cenUm.iops_total[dataset_write_cenUm.iops_total != 0] = (1/dataset_write_cenUm.iops_total)


In [ ]:
teste = dataset_cenUm.query('repetition==2 & individual_total == "I" & operation == "W" & pattern == "R" & work_outstd == 4 & capacity == "1TBHDD" & object_size == "4KB"')
if (not teste.empty):
    print ('eh')
else:
    x = teste.iops_total.get_values()[0]
    print (x)
    

    
           
#teste.iops_total
#teste.info()
#print (teste.iops_total.to_string(index=False))

In [ ]:
#df = dataset.query('object_size == "128KB" & capacity == "120GBSSD" & scenario == 2')

#df.shape

dataset.info()

#dataset_write_cenUm.shape
